In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=cecac0de32690010a31ca6a365efea194a507791810ba947c75be7b2c34da350
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


In [1]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("I feel a energetic") # run annotation over a sentence

ModuleNotFoundError: ignored

In [ ]:
!pip install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import conllu
import torch
import torchtext
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence




def build_vocabularies(data_file):
    pos_tags = [ "ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"]
    pos_tags = [[i] for i in pos_tags]
    assert(len(pos_tags) == 17)
    vocab_pos_tags = torchtext.vocab.build_vocab_from_iterator(pos_tags)
    vocab_pos_tags.append_token("pad")
    
    dep_typ = ["nsubj", "obj", "iobj", "csubj", "ccomp", "xcomp", "obl", "vocative", "expl", "dislocated", "advcl", "advmod", "discourse" , "aux", "cop", "mark",	 "nmod", "appos", "nummod", "acl", "amod", "det", "clf", "case", "conj", "cc",	 "fixed", "flat", "compound", "list", "parataxis", "orphan", "goeswith", "reparandum", "punct", "root", "dep"]
    dep_typ = [[i] for i in dep_typ]
    assert len(dep_typ) == 37
    vocab_deptyp = torchtext.vocab.build_vocab_from_iterator(dep_typ)
    vocab_deptyp.append_token("pad")

    words = [["<unk>"], ["<unk>"]]
    TokenLists = conllu.parse_incr(open(data_file, "r", encoding="utf-8"))
    for TokenList in TokenLists:
        for token in TokenList:
            words.append([token["form"]])
    vocab_words = torchtext.vocab.build_vocab_from_iterator(words, min_freq=2)
    vocab_words.set_default_index(vocab_words["<unk>"])
    #vocab_words.append_token("pad")
    return vocab_pos_tags, vocab_deptyp, vocab_words


class DepData(Dataset):
    def __init__(self, data_file, vocab_pos_tags, vocab_deptyp, vocab_words):
        super().__init__()
        self.vocab_pos_tags = vocab_pos_tags
        self.vocab_deptyp = vocab_deptyp
        self.vocab_words = vocab_words

        Data = []
        TokenLists = conllu.parse_incr(open(data_file, "r", encoding="utf-8"))
        for TokenList in TokenLists:
            data = []
            for token in TokenList:
                if type(token["id"]) == int:
                    data.append([token["id"], token["form"], token["upos"], token["head"], token["deprel"].split(":")[0]])
                else:
                    pass
            #data = torch.LongTensor(data)
            Data.append(data)
        self.Data = Data

    def __len__(self):
        return len(self.Data)
    
    def __getitem__(self, index):
        return self.Data[index]

 
def custom_collate(batch):
    return pad_sequence(batch, batch_first=True)

vocab_pos_tags, vocab_deptyp, vocab_words = build_vocabularies("./en_ewt-ud-test.conllu")

data_t = DepData("./en_ewt-ud-test.conllu", vocab_pos_tags, vocab_deptyp, vocab_words)
#data_loader = DataLoader(data_t, batch_size=32, shuffle=True, collate_fn=custom_collate)


# print(vocab_deptyp["pad"], vocab_pos_tags["pad"], vocab_words["pad"])
# data_2 = DepData("UD_English-EWT-master/en_ewt-ud-test.conllu", vocab_pos_tags, vocab_deptyp, vocab_words)
# data_3 = DepData("UD_English-EWT-master/en_ewt-ud-dev.conllu",  vocab_pos_tags, vocab_deptyp, vocab_words)
# for x in data_3_loader:
#     pass

In [ ]:
print(data_t[1])


[[1, 'What', 'PRON', 0, 'root'], [2, 'if', 'SCONJ', 4, 'mark'], [3, 'Google', 'PROPN', 4, 'nsubj'], [4, 'expanded', 'VERB', 1, 'advcl'], [5, 'on', 'ADP', 15, 'case'], [6, 'its', 'PRON', 15, 'nmod'], [7, 'search', 'NOUN', 9, 'compound'], [8, '-', 'PUNCT', 9, 'punct'], [9, 'engine', 'NOUN', 15, 'compound'], [10, '(', 'PUNCT', 9, 'punct'], [11, 'and', 'CCONJ', 13, 'cc'], [12, 'now', 'ADV', 13, 'advmod'], [13, 'e-mail', 'NOUN', 9, 'conj'], [14, ')', 'PUNCT', 15, 'punct'], [15, 'wares', 'NOUN', 4, 'obl'], [16, 'into', 'ADP', 22, 'case'], [17, 'a', 'DET', 22, 'det'], [18, 'full', 'ADV', 20, 'advmod'], [19, '-', 'PUNCT', 20, 'punct'], [20, 'fledged', 'ADJ', 22, 'amod'], [21, 'operating', 'NOUN', 22, 'compound'], [22, 'system', 'NOUN', 4, 'obl'], [23, '?', 'PUNCT', 4, 'punct']]


In [ ]:
import re
sentences=[]

with open('./en_ewt-ud-test.conllu', 'r') as f:
  for line in f:
    tokens=line.split(" ")
    if tokens[0]=="#":
      if tokens[1]=="newdoc" or tokens[1]=="sent_id" or tokens[1]=="newpar":continue
      line=re.sub("# text = ","",line)
      sentences.append(line)
total=0
uas=0
las=0
for i in range(0,len(sentences)):
  pred=nlp(sentences[i]).to_dict()
  target=data_t[i]
  if len(pred[0])==len(target):
    print(f"{len(pred[0])} {len(target)} {sentences[i]}")
    for j in range(len(pred[0])):
      if pred[0][j]["text"]==target[j][1]:
        if pred[0][j]["head"]==target[j][3]:
          uas+=1
          if pred[0][j]["deprel"]==target[j][4]:
            las+=1
        total+=1 #its not +1 , its +len(sentence)


  #compare pred, target 
  #target - [id, word, pos, head,rel_type]





In [ ]:
# print(uas)
# print(las)
print('LAS score:')
print((las/total)*100)
# print(total)
print('UAS score:')
print((uas/total)*100)

LAS score:
89.1461255617595
UAS score:
91.31145308457033
